In [1]:
# 引用設定
import variable

# 服務名稱
bucketName = variable.bucketName
queueName = variable.queueName
tableName = variable.tableName

# 服務連線
s3_resource = variable.s3_resource
sqs_client = variable.sqs_client
dynamoDB_resource = variable.dynamoDB_resource

# 使用的參數
# http://s3.vcloudlab.pro:4569/s3/vcloudlab_bucket/
baseUrl = variable.baseURL

In [ ]:
# 創建 s3 Bucket
s3_resource.create_bucket( Bucket = bucketName )

# 創建 sqs Long Polling Queue
sqs_client.create_queue( 
    QueueName = queueName,
    Attributes = {'ReceiveMessageWaitTimeSeconds': '20'}
)

# 創建 DynamoDB Table
vlabTable = dynamoDB_resource.create_table(
    TableName = tableName,
    
    KeySchema = [
      {
          'AttributeName': 'id',
          'KeyType': 'HASH'
      },
        
    ],
    
    AttributeDefinitions = [
      {
          'AttributeName': 'id',
          'AttributeType': 'S'
      },
    ],
    
    ProvisionedThroughput = {
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)
vlabTable.meta.client.get_waiter('table_exists').wait(TableName=tableName)

In [2]:
# 若是創建過服務就直接調用
# 調用 SQS
vlabQueues = sqs_client.list_queues( QueueNamePrefix = queueName )
queue_url = vlabQueues['QueueUrls'][0]
print(queue_url)

# 調用 dynamoDB Table
vlabTable = dynamoDB_resource.Table(tableName)
print(vlabTable)

http://sqs.vcloudlab.pro:9324/queue/vcloudlab_sqs_queue
dynamodb.Table(name='vcloudlab_picture')


In [7]:
import datetime
# 透過 SQS  將 filename 抓取
while True:
    messages = sqs_client.receive_message(
        QueueUrl = queue_url,
        MaxNumberOfMessages = 10,
        WaitTimeSeconds = 20
    )

    if 'Messages' in messages:
        for message in messages['Messages']: 
            print(message['Body'])
            filename = message['Body']
            # 先判斷目前的 id
            getId = vlabTable.item_count
            setId = getId+1

            # 將取得的 filename 存入 dynamodb
            time = datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
            vlabTable.put_item(
                Item = {
                    'id': str(setId),
                    'datetime': str(time),
                    'fileurl': str(baseUrl)+str(filename),
                    'filename': str(filename),
                }
            )

            sqs_client.delete_message(
                QueueUrl = queue_url,
                ReceiptHandle=message['ReceiptHandle']
            )
    else:
        print('Queue is now empty')
        #break

Queue is now empty
ROCKBOUND_NEIGHBORS_01.jpg
ROCKBOUND_NEIGHBORS_02.jpg
ROCKBOUND_NEIGHBORS_03.jpg
Queue is now empty
Queue is now empty
Queue is now empty
Queue is now empty
Queue is now empty
Queue is now empty
Queue is now empty
Queue is now empty
Queue is now empty
Queue is now empty
Queue is now empty
Queue is now empty
Queue is now empty
Queue is now empty
Queue is now empty


KeyboardInterrupt: 

In [ ]:
response = vlabTable.get_item(
    Key = {
        'id': '1',
    }
)

item = response['Item']
print(item)

In [5]:
sqs_client.set_queue_attributes(
    QueueUrl = queue_url,
    Attributes = {'ReceiveMessageWaitTimeSeconds': '20'}
)

{'ResponseMetadata': {'RequestId': '00000000-0000-0000-0000-000000000000',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'akka-http/10.0.2',
   'date': 'Mon, 28 Jan 2019 05:28:11 GMT',
   'content-type': 'text/plain; charset=UTF-8',
   'content-length': '264'},
  'RetryAttempts': 0}}